# ***PyTorch Native Distributed Training with Amazon SageMaker***

In [1]:
# !pip install sagemaker --upgrade -q
# !pip install ipywidgets -q

**Step 1:** Import essentials packages, start a sagemaker session and specify the bucket name you created in the pre-requsites section of this workshop.

In [2]:
import os
import boto3
import time
import numpy as np
import sagemaker

sess = boto3.Session()
sm   = sess.client('sagemaker')
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

bucket_name    = sagemaker_session.default_bucket()
jobs_folder    = 'jobs'
dataset_folder = 'datasets'

![](https://miro.medium.com/max/1000/0*GRfvsrvtfpRm400-)

#### Prepare the training data
The CIFAR-10 dataset is a subset of the 80 million tiny images dataset. It consists of 60,000 32x32 color images in 10 classes, with 6,000 images per class.

In [3]:
import torchvision
cifar10_dataset = torchvision.datasets.CIFAR10('cifar10-dataset', 
                                     train=True, 
                                     download=True)

datasets = sagemaker_session.upload_data(path='cifar10-dataset', 
                                         key_prefix=f'{dataset_folder}/cifar10-dataset')

datasets

Files already downloaded and verified


's3://sagemaker-us-west-2-453691756499/datasets/cifar10-dataset'

**Step 2:** Specify hyperparameters, instance type and number of instances to distribute training to. 

In [4]:
job_name   = f'pytorch-native-dist-{time.strftime("%Y-%m-%d-%H-%M-%S-%j", time.gmtime())}'
output_path = f's3://{bucket_name}/{jobs_folder}'

hyperparameters = {'epochs'       : 20, 
                   'lr'           : 0.01,
                   'momentum'     : 0.9,
                   'batch-size'   : 64,
                   'model-type'   : 'custom',
                   'backend'      : 'gloo'}

In [5]:
from sagemaker.pytorch import PyTorch
estimator = PyTorch(entry_point         = 'cifar10-distributed-native-cpu.py', 
                           source_dir           = 'code',
                           output_path          = output_path + '/',
                           code_location        = output_path,
                           role                 = role,
                           instance_count       = 2,
                           instance_type        = 'ml.c5.2xlarge',
                           framework_version    = '1.8.0', 
                           py_version           = 'py3',
                           hyperparameters      = hyperparameters)

**Step 4:** Specify dataset locations in Amazon S3 and then call the fit function.

In [ ]:
estimator.fit({'train': datasets}, 
              job_name=job_name, 
              wait=True)

2022-06-21 09:45:10 Starting - Starting the training job...
2022-06-21 09:45:33 Starting - Preparing the instances for trainingProfilerReport-1655804710: InProgress
......
2022-06-21 09:46:34 Downloading - Downloading input data...
2022-06-21 09:46:58 Training - Downloading the training image...
2022-06-21 09:47:34 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-06-21 09:47:25,631 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-06-21 09:47:25,633 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-06-21 09:47:25,646 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-06-21 09:47:25,652 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-06-21 09:47:25,938 sagemaker-training-toolkit INFO     No GPUs 

In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorchModel

model = PyTorchModel(
    model_data=estimator.model_data,
    source_dir="code",
    entry_point="inference.py",
    role=role,
    framework_version="1.6.0",
    py_version="py3",
)

predictor = model.deploy(initial_instance_count=1, instance_type="ml.c5.xlarge")

In [ ]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

classes = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

def test_data_loader():
    transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), 
                                                torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    testset = torchvision.datasets.CIFAR10(root="cifar10-dataset", 
                                           train=False, 
                                           download=False, 
                                           transform=transform)                                            
    return torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

def show_img(img):
    """displays an image"""
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    

In [ ]:
# get some test images
dataiter = iter(test_data_loader())
images, labels = dataiter.next()

# print images, labels, and predictions
show_img(torchvision.utils.make_grid(images))
print("GroundTruth: ", " ".join("%4s" % classes[labels[j]] for j in range(4)))
outputs = predictor.predict(images)
_, predicted = torch.max(torch.from_numpy(np.array(outputs)), 1)

print("Predicted:   ", " ".join("%4s" % classes[predicted[j]] for j in range(4)))